In [ ]:
import json
import os
from typing import Dict, List, Optional, Tuple
from datetime import datetime


class YearlyKeywordChangeAnalyzer:
    """1년간 키워드 변화를 분석하는 클래스"""
    
    def __init__(self, similarity_threshold: float = 0.3):
        self.similarity_threshold = similarity_threshold
    
    def find_similar_keyword(self, current_keyword: Dict, previous_keywords: List[Dict]) -> Optional[Dict]:
        """현재 키워드와 비슷한 이전 키워드를 찾는 함수"""
        current_phrase = current_keyword['phrase'].lower()
        current_merged = current_keyword.get('merged_keywords', [])
        
        # 1. 정확히 같은 키워드 찾기
        for prev_keyword in previous_keywords:
            if prev_keyword['phrase'].lower() == current_phrase:
                return prev_keyword
        
        # 2. merged_keywords에서 매칭 찾기
        for prev_keyword in previous_keywords:
            prev_merged = prev_keyword.get('merged_keywords', [])
            
            # 현재 키워드의 merged_keywords와 이전 키워드의 phrase 비교
            if any(merged.lower() == prev_keyword['phrase'].lower() for merged in current_merged):
                return prev_keyword
            
            # 현재 키워드의 phrase와 이전 키워드의 merged_keywords 비교
            if any(merged.lower() == current_phrase for merged in prev_merged):
                return prev_keyword
            
            # merged_keywords 간 교집합 확인
            intersection = [curr for curr in current_merged 
                          if any(prev.lower() == curr.lower() for prev in prev_merged)]
            if intersection:
                return prev_keyword
        
        # 3. 부분 매칭 (키워드 일부가 포함되는 경우)
        for prev_keyword in previous_keywords:
            prev_phrase = prev_keyword['phrase'].lower()
            if current_phrase in prev_phrase or prev_phrase in current_phrase:
                return prev_keyword
        
        return None
    
    def get_change_status(self, current: int, previous: int) -> str:
        """변화 상태를 결정하는 함수"""
        if previous == 0:
            return "신규"
        change = current - previous
        if change > 0:
            return "증가"
        elif change < 0:
            return "감소"
        else:
            return "동일"
    
    def analyze_monthly_changes(self, current_data: Dict, previous_data: Optional[Dict], top_n: int = 10) -> List[Dict]:
        """월별 키워드 변화를 분석하는 함수"""
        # 현재 달 상위 N개 키워드
        current_top = current_data['keywords'][:min(top_n, len(current_data['keywords']))]
        
        results = []
        
        for index, current_keyword in enumerate(current_top):
            # 이전 달 데이터가 있는 경우에만 비교
            if previous_data:
                matched_keyword = self.find_similar_keyword(current_keyword, previous_data['keywords'])
                previous_doc_count = matched_keyword['doc_count'] if matched_keyword else 0
                previous_score = matched_keyword['score'] if matched_keyword else 0
                matched_phrase = matched_keyword['phrase'] if matched_keyword else "없음"
            else:
                # 첫 번째 월이거나 이전 데이터가 없을 때
                previous_doc_count = 0
                previous_score = 0
                matched_phrase = "없음"
            
            # 변화량 계산
            change = current_keyword['doc_count'] - previous_doc_count
            
            # 변화율 계산
            if previous_doc_count > 0:
                change_percent = round(((current_keyword['doc_count'] - previous_doc_count) / previous_doc_count) * 100, 1)
            else:
                change_percent = "신규"
            
            result = {
                'rank': index + 1,
                'current_phrase': current_keyword['phrase'],
                'current_doc_count': current_keyword['doc_count'],
                'current_score': current_keyword['score'],
                'previous_doc_count': previous_doc_count,
                'previous_score': previous_score,
                'change': change,
                'change_percent': change_percent,
                'matched_phrase': matched_phrase,
                'status': self.get_change_status(current_keyword['doc_count'], previous_doc_count)
            }
            
            results.append(result)
        
        return results
    
    def load_json_file(self, file_path: str) -> Optional[Dict]:
        """JSON 파일을 로드하는 함수"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
            return None
        except json.JSONDecodeError:
            print(f"JSON 파일을 읽는데 오류가 발생했습니다: {file_path}")
            return None
    
    def save_results_to_json(self, results: Dict, output_path: str):
        """결과를 JSON 파일로 저장"""
        try:
            with open(output_path, 'w', encoding='utf-8') as file:
                json.dump(results, file, ensure_ascii=False, indent=2)
            print(f"결과가 저장되었습니다: {output_path}")
        except Exception as e:
            print(f"파일 저장 중 오류가 발생했습니다: {e}")
    
    def analyze_yearly_changes(self, year: int, input_dir: str, output_path: str, top_n: int = 10):
        """1년간의 키워드 변화를 분석하고 저장"""
        yearly_results = {
            'year': year,
            'analysis_date': datetime.now().isoformat(),
            'monthly_changes': {},
            'yearly_summary': {}
        }
        
        previous_month_data = None
        all_monthly_stats = []
        
        print(f"=== {year}년 연간 키워드 변화 분석 시작 ===\n")
        
        # 1월부터 12월까지 순차 분석
        for month in range(1, 13):
            month_str = f"{month:02d}"
            file_path = os.path.join(input_dir, f"{year}_{month_str}_keyword_grouped.json")
            
            print(f"[{month}월] 분석 중... -> {file_path}")
            
            # 현재 월 데이터 로드
            current_data = self.load_json_file(file_path)
            if current_data is None:
                print(f"  ⚠️  {month}월 데이터를 찾을 수 없습니다. 건너뜁니다.\n")
                # 이전 달 데이터는 그대로 유지 (다음 달 비교를 위해)
                continue

            # ✅ 전월 데이터 결정 로직 (연도 경계 처리 포함)
            if month == 1:
                # 전년도 12월 파일 시도
                prev_file = os.path.join(input_dir, f"{year-1}_12_keyword_grouped.json")
                prev_data = self.load_json_file(prev_file)
                if prev_data is None:
                    print(f"  ℹ️  전년도(#{year-1}) 12월 데이터가 없어 1월은 '신규'로 처리됩니다.")
            else:
                # 직전 반복에서 저장해둔 데이터 사용
                prev_data = previous_month_data
            
            # 월별 변화 분석
            monthly_changes = self.analyze_monthly_changes(current_data, prev_data, top_n)
            
            # 월별 통계 계산
            monthly_stats = self.calculate_monthly_stats(monthly_changes, month, prev_data is not None)
            all_monthly_stats.append(monthly_stats)
            
            # 결과 저장
            yearly_results['monthly_changes'][f"{month:02d}"] = {
                'month': month,
                'current_data_info': {
                    'year': current_data.get('year', year),
                    'month': current_data.get('month', month),
                    'total_keywords': len(current_data['keywords'])
                },
                'changes': monthly_changes,
                'stats': monthly_stats
            }
            
            print(f"  ✓ {month}월 분석 완료 (키워드 {len(monthly_changes)}개)\n")
            
            # 다음 반복을 위해 현재 데이터를 이전 데이터로 설정
            previous_month_data = current_data
        
        # 연간 요약 통계 계산
        yearly_results['yearly_summary'] = self.calculate_yearly_summary(all_monthly_stats)
        
        # 결과 저장
        self.save_results_to_json(yearly_results, output_path)
        
        # 요약 출력
        self.print_yearly_summary(yearly_results)
        
        return yearly_results
    
    def calculate_monthly_stats(self, changes: List[Dict], month: int, has_previous: bool) -> Dict:
        """월별 통계 계산"""
        if not has_previous:
            # 첫 번째 달은 (전월 데이터 없으면) 모든 키워드가 신규
            return {
                'total_keywords': len(changes),
                'new_keywords': len(changes),
                'increased': 0,
                'decreased': 0,
                'unchanged': 0,
                'biggest_increase': None,
                'biggest_decrease': None,
                'avg_doc_count': sum(c['current_doc_count'] for c in changes) / len(changes) if changes else 0
            }
        
        new_keywords = len([c for c in changes if c['status'] == '신규'])
        increased = len([c for c in changes if c['status'] == '증가'])
        decreased = len([c for c in changes if c['status'] == '감소'])
        unchanged = len([c for c in changes if c['status'] == '동일'])
        
        # 가장 큰 변화
        increases = [c for c in changes if c['change'] > 0]
        decreases = [c for c in changes if c['change'] < 0]
        
        biggest_increase = max(increases, key=lambda x: x['change']) if increases else None
        biggest_decrease = min(decreases, key=lambda x: x['change']) if decreases else None
        
        return {
            'total_keywords': len(changes),
            'new_keywords': new_keywords,
            'increased': increased,
            'decreased': decreased,
            'unchanged': unchanged,
            'biggest_increase': {
                'phrase': biggest_increase['current_phrase'],
                'change': biggest_increase['change'],
                'change_percent': biggest_increase['change_percent']
            } if biggest_increase else None,
            'biggest_decrease': {
                'phrase': biggest_decrease['current_phrase'],
                'change': biggest_decrease['change'],
                'change_percent': biggest_decrease['change_percent']
            } if biggest_decrease else None,
            'avg_doc_count': sum(c['current_doc_count'] for c in changes) / len(changes) if changes else 0
        }
    
    def calculate_yearly_summary(self, monthly_stats: List[Dict]) -> Dict:
        """연간 요약 통계 계산"""
        if not monthly_stats:
            return {}
        
        total_months = len(monthly_stats)
        
        return {
            'analyzed_months': total_months,
            'avg_keywords_per_month': sum(s['total_keywords'] for s in monthly_stats) / total_months,
            'total_new_keywords': sum(s['new_keywords'] for s in monthly_stats),
            'avg_new_keywords_per_month': sum(s['new_keywords'] for s in monthly_stats) / total_months,
            'most_volatile_month': max(monthly_stats, key=lambda x: x['increased'] + x['decreased'])['total_keywords'] if monthly_stats else None,
            'avg_doc_count_trend': [round(s['avg_doc_count'], 1) for s in monthly_stats]
        }
    
    def print_yearly_summary(self, yearly_results: Dict):
        """연간 요약 결과 출력"""
        year = yearly_results['year']
        summary = yearly_results['yearly_summary']
        
        print(f"\n{'='*50}")
        print(f"  {year}년 키워드 변화 분석 요약")
        print(f"{'='*50}")
        print(f"분석 완료 월수: {summary['analyzed_months']}개월")
        print(f"월평균 분석 키워드: {summary['avg_keywords_per_month']:.1f}개")
        print(f"연간 신규 키워드 총합: {summary['total_new_keywords']}개")
        print(f"월평균 신규 키워드: {summary['avg_new_keywords_per_month']:.1f}개")
        
        print(f"\n월별 평균 문서수 추이:")
        for i, avg_count in enumerate(summary['avg_doc_count_trend'], 1):
            print(f"  {i:2d}월: {avg_count}")
        
        print(f"\n분석 완료 시간: {yearly_results['analysis_date']}")


def main():
    """메인 실행 함수"""
    # 분석기 초기화
    analyzer = YearlyKeywordChangeAnalyzer(similarity_threshold=0.3)
    
    # 설정값들
    year = 2016
    input_directory = "/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results_cluster"
    output_file = f"/home/ds4_sia_nolb/#FINAL_POLARIS/07_Visualization/1.top_keyword/re_monthly_keyword_change_volume/yearly_analysis_{year}.json"
    top_keywords = 10
    
    # 1년간 분석 실행
    results = analyzer.analyze_yearly_changes(
        year=year,
        input_dir=input_directory,
        output_path=output_file,
        top_n=top_keywords
    )
    
    print(f"\n🎉 {year}년 연간 키워드 변화 분석이 완료되었습니다!")
    print(f"결과 파일: {output_file}")


if __name__ == "__main__":
    main()


=== 2016년 연간 키워드 변화 분석 시작 ===

[1월] 분석 중... -> /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/monthly_results_cluster/2016_01_keyword_grouped.json
파일을 찾을 수 없습니다: /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/monthly_results_cluster/2015_12_keyword_grouped.json
  ℹ️  전년도(#2015) 12월 데이터가 없어 1월은 '신규'로 처리됩니다.
  ✓ 1월 분석 완료 (키워드 10개)

[2월] 분석 중... -> /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/monthly_results_cluster/2016_02_keyword_grouped.json
  ✓ 2월 분석 완료 (키워드 10개)

[3월] 분석 중... -> /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/monthly_results_cluster/2016_03_keyword_grouped.json
  ✓ 3월 분석 완료 (키워드 10개)

[4월] 분석 중... -> /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/monthly_results_cluster/2016_04_keyword_grouped.json
  ✓ 4월 분석 완료 (키워드 10개)

[5월] 분석 중... -> /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/monthly_results_cluster/2016_05_keyword_grouped.json
  ✓ 5월 분석 완료 (키워드 10개)

[6월] 분석 중... -> /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/monthly_results_cluster/2016_06